## Нахождение цен и ее оптимизация для европейских и американских опционов в биномиальной модели.

In [1]:
import numpy as np

### 1. Европейский алгоритм за $O(n^2)$

In [2]:
def eur_alg_square(r, u, d, s, n, g):
    p_neut = (1 + r - d)/(u - d)
    q_neut = 1 - p_neut
    v = np.zeros((n + 1, n + 1))
    
    for i in range(0, n+1):
        v[n][i] = max(g((u ** i) * (d ** (n - i)) * s), 0)
    
    for period in range(n-1, -1, -1):
        for j in range(0, period + 1):
            v[period][j] = (1 / (1 + r)) * (p_neut * v[period + 1][j] + q_neut * v[period + 1][j + 1])
    return v[0][0]

Работа для примера из текстового файла:

In [3]:
def geur(s):
    return 5-s

eur_alg_square(1/4, 2, 1/2, 4, 2, geur)

0.96

### 2. Европейский алгоритм за $O(n)$

In [4]:
def factorial(n):
    if (n == 0):
        return 1
    else:
        return n * factorial(n-1)
    
def bin_coef(n, m):
    return factorial(n) / ((factorial(n - m)) * factorial(m))

In [5]:
def eur_alg_linear(r, u, d, s, n, g):
    p_neut = (1 + r - d)/(u - d)
    q_neut = 1 - p_neut
    v = np.zeros(n + 1)
    
    for i in range(0, n + 1):
        v[i] = max(g((u ** i) * (d ** (n - i)) * s), 0)
    
    v_0 = 0
    for i in range(0, n + 1):
        v_0 += (p_neut ** (n-i)) * (q_neut ** i) * bin_coef(n, i) * v[n-i]
    
    v_0 *= (1 / (1 + r)) ** n
    
    return v_0

Работа для примера из текстового файла:

In [6]:
def geur(s):
    return 5-s

res = eur_alg_linear(1/4, 2, 1/2, 4, 2, geur)
print(float('{:.7f}'.format(res)))

0.96


### 3. Американский алгоритм за $O(n^2)$

In [7]:
def amer_alg(r, u, d, s, n, g):
    p_neut = (1 + r - d)/(u - d)
    q_neut = 1 - p_neut
    v = np.zeros((n + 1, n + 1))
    
    for i in range(0, n+1):
        v[n][i] = max(g((u ** i) * (d ** (n - i)) * s), 0)
    
    for period in range(n-1, -1, -1):
        for j in range(0, period + 1):
            v[period][j] = max(
                g((u ** j) * (d ** (period - j)) * s), 
                (1 / (1 + r)) * (p_neut * v[period + 1][j] + q_neut * v[period + 1][j + 1])
            )
    return v[0][0]

Для примера из текстового файла:

In [53]:
def geur(s):
    return 5-s

res2 = amer_alg(1/4, 2, 1/2, 4, 2, geur)
print(float('{:.7f}'.format(res2)))

1.36


### 4. Оптимизированный американский алгоритм

В функции ниже используется, что рациональная выпуклость функции + ее непрерывность дают выпуклость этой функции.

In [13]:
# проверка функции на выпуклость
def is_convex(g, s, u, n):
    is_conv = True
    
    for i in range(10000):
        s_1, s_2 = np.random.uniform(low = 0.0, high = u ** n * s, size = 2)
        if (g((s_1 + s_2) / 2) - (g(s_1) + g(s_2))/2 > 1e-7):
            is_conv = False
            #print(s_1, s_2)
            
    return is_conv

# специально для путей в бин. модели
def is_convex_extra(g, min_val, max_val):
    is_conv = True
    
    range_size = (int)(max_val - min_val) * 10
    
    for i in range(range_size):
        s_1, s_2 = np.random.uniform(low = min_val, high = max_val, size = 2)
        if (g((s_1 + s_2) / 2) - (g(s_1) + g(s_2))/2 > 1e-7):
            is_conv = False
            
    return is_conv

# проверка на выпуклость только в вершинах дерева
def is_convex_fast(g, s, u, d, n, cur_period):
    is_conv = True
    
    node_values = []
    for period in range(cur_period, n+1):
        for j in range(0, period+1):
            node_values.append(s * (u ** j) * (d ** (period - j)))
    
    for value_1 in node_values:
        for value_2 in node_values:
            if (g((value_1 + value_2) / 2) - (g(value_1) + g(value_2))/2 > 1e-7):
                is_conv = False
                
    return is_conv

In [14]:
# Пример для проверки: функция выплат американского опциона колл со страйком K
def amer(s):
    return s-5

print(is_convex(amer, 4, 2, 2))
print(is_convex_fast(amer, 4, 2, 1/2, 2, 0))

True
True


In [26]:
def amer_alg_opt(r, u, d, s, n, g):
    
    # случай, когда g - выпуклая
    if (is_convex_fast(g, s, u, d, n, 0) and g(0) == 0):
        return eur_alg_linear(r, u, d, s, n, g)
    
    # случай, когда g - частично выпуклая
    if (g(0) == 0):
        p_neut = (1 + r - d)/(u - d)
        q_neut = 1 - p_neut
        v = np.zeros((n + 1, n + 1))
        
        # значения в последнем периоде
        s_last = []
        for i in range(n+1):
            s_last.append(s * (u ** j) * (d ** (n - j)))
    
        for i in range(0, n+1):
            v[n][i] = max(g((u ** i) * (d ** (n - i)) * s), 0)
            
        is_partly_convex = False
        convex_indexes = []
        convex_period = 0
        
        # ищем самый ранний период, в котором можно упростить
        for period in range(0, n):
            for j in range(0, period + 1):
                if (is_convex_fast(g, s, u, d, n, period)):
                    is_partly_convex = True
                    convex_indexes.append(j)
                    convex_period = period
            if(is_partly_convex):  
                break
        
        if(is_partly_convex):
            for j in range(0, convex_period + 1):
                if (j in convex_indexes):
                    s_current = (u ** j) * (d ** (period - j)) * s
                    v[period][j] = eur_alg_linear(r, u, d, s_current, n - period, g)
                else:
                    s_current = (u ** j) * (d ** (period - j)) * s
                    v[period][j] = amer_alg(r, u, d, s_current, n - period, g)
                    
            for period in range(convex_period - 1, -1, -1):
                for j in range(0, period + 1):
                    v[period][j] = v[period][j] = max(
                g((u ** j) * (d ** (period - j)) * s), 
                (1 / (1 + r)) * (p_neut * v[period + 1][j] + q_neut * v[period + 1][j + 1])
            )
            
        else:
            v[0][0] = amer_alg(r, u, d, s, n, g)
        
        return v[0][0]
    
    else:
        return amer_alg(r, u, d, s, n, g)

In [27]:
# опцион пут со страйком 2
def amer_put_ex(s):
    return max(0, 2-s)

amer_alg_opt(1/4, 2, 1/2, 4, 4, amer_put_ex)

0.21120000000000005

Разберем другую вариацию алгоритма. В текстовом файле была теорема:

* Рассмотрим биномиальную модель с $n$ периодами, где $0 < d < 1+r < u$ и процентная ставка неотрицательна ($r \geq 0 $). Пускай функция выплат (в американской модели) $g(s)$ - выпуклая и $g(0) = 0$. Тогда стоимость данной ценной бумаги в нулевой момент времени равна стоимости ценной бумаги в европейской модели с той же функцией выплат и сроком погашения $n$. 

Теорема останется верна, если добавим условие "$g$ - неубывающая".

In [28]:
# проверка функции на неубываемость
def is_nondecreasing(g, s, u, n):
    is_nondec = True
    
    range_size = u ** n * s * 100
    
    for i in range(range_size):
        s_1, s_2 = np.random.uniform(low = 0.0, high = u ** n * s, size = 2)
        if ((g(s_1) - g(s_2)) * (s_1 - s_2) < 0):
            is_nondec = False
            
    return is_nondec

# специально для путей в бин. модели
def is_nondecreasing_extra(g, min_val, max_val):
    is_nondec = True
    
    range_size = (int)(max_val - min_val) * 10
    
    for i in range(range_size):
        s_1, s_2 = np.random.uniform(low = min_val, high = max_val, size = 2)
        if ((g(s_1) - g(s_2)) * (s_1 - s_2) < 0):
            is_nondec = False
            
    return is_nondec

# в узлах дерева
def is_nondecreasing_fast(g, s, u, d, n, cur_period):
    is_nondec = True
    
    node_values = []
    for period in range(cur_period, n+1):
        for j in range(0, period+1):
            node_values.append(s * (u ** j) * (d ** (period - j)))
    
    for value_1 in node_values:
        for value_2 in node_values:
            if ((g(value_1) - g(value_2)) * (value_1 - value_2) < 0):
                is_nondec = False
                
    return is_nondec

In [39]:
def amer_alg_opt_2(r, u, d, s, n, g):
    
    # случай, когда g - выпуклая
    if (is_convex(g, s, u, n) and g(0) == 0):
        return eur_alg_linear(r, u, d, s, n, g)
    
    # случай, когда g - частично выпуклая
    if (g(0) == 0):
        p_neut = (1 + r - d)/(u - d)
        q_neut = 1 - p_neut
        v = np.zeros((n + 1, n + 1))
        
        # значения в последнем периоде
        s_last = []
        for j in range(n+1):
            s_last.append(s * (u ** j) * (d ** (n - j)))
    
        for i in range(0, n+1):
            v[n][i] = max(g((u ** i) * (d ** (n - i)) * s), 0)
            
        is_partly_convex = False
        convex_indexes = []
        convex_period = 0
        
        # ищем самый ранний период, в котором можно упростить
        for period in range(0, n):
            for j in range(0, period + 1):
                s_current = (u ** j) * (d ** (period - j)) * s
                max_current = s_current * (u ** (n - j))
                min_current = s_current * (d ** (n - j))
                if(is_convex_fast(g, s, u, d, n, period) and is_nondecreasing_fast(g, s, u, d, n, period)):
                    is_partly_convex = True
                    convex_indexes.append(j)
                    convex_period = period
                    #print(min_current, max_current)
                if(is_partly_convex):  
                    break
    
        #print(convex_period)
        #print(convex_indexes)
    
        if(is_partly_convex):
            for j in range(0, convex_period + 1):
                if (j in convex_indexes):
                    s_current = (u ** j) * (d ** (period - j)) * s
                    v[period][j] = eur_alg_linear(r, u, d, s_current, n - period, g)
                else:
                    s_current = (u ** j) * (d ** (period - j)) * s
                    v[period][j] = amer_alg(r, u, d, s_current, n - period, g)
                    
            for period in range(convex_period - 1, -1, -1):
                for j in range(0, period + 1):
                    v[period][j] = max(
                    g((u ** j) * (d ** (period - j)) * s), 
                    (1 / (1 + r)) * (p_neut * v[period + 1][j] + q_neut * v[period + 1][j + 1])
                )
            
        else:
            v[0][0] = amer_alg(r, u, d, s, n, g)
            
        return v[0][0]    
    
    else:
        return amer_alg(r, u, d, s, n, g)

In [40]:
# опцион пут со страйком 5
def amer_put_ex(s):
    return max(0, 5-s)

amer_alg_opt_2(1/4, 2, 1/2, 2, 4, amer_put_ex)

3.0

In [41]:
def amer_put_ex33(s):
    return max(0, 12-s)

amer_alg_opt_2(1/4, 2, 1/2, 20, 10, amer_put_ex33)

1.7168134144000005

In [42]:
# для сравнения
amer_alg_opt(1/4, 2, 1/2, 20, 10, amer_put_ex33)

1.7168134144000005

### 5. Европейский опцион

In [43]:
def eur_call(K, s):
    return (s - K)

def eur_put(K, s):
    return (K - s)

Функции для вычисления цены европейских опционов колл и пут (работают за $O(n)$):

In [44]:
def eur_option_call_price(r, u, d, s, n, K):
    
    def g(s):
        return eur_call(K, s)
    
    v_0 = eur_alg_linear(r, u, d, s, n, g)
    
    return v_0


def eur_option_put_price(r, u, d, s, n, K):
    
    def g(s):
        return eur_put(K, s)
    
    v_0 = eur_alg_linear(r, u, d, s, n, g)
    
    return v_0

И снова, проверим для примера из текстового файла:

In [45]:
res3 = eur_option_put_price(1/4, 2, 1/2, 4, 2, 5)
print(float('{:.7f}'.format(res3)))

0.96


### 6. Американский опцион

In [46]:
def amer_call(K, s):
    return (s - K)

def amer_put(K, s):
    return (K - s)

В тексте было доказано, что для американского опциона колл функция $g$ - выпуклая, так что по Теореме получим, что цену американского опциона колл можно посчитать за $O(n)$ и она будет равна цене европейского опциона колл.

In [47]:
def amer_option_call_price(r, u, d, s, n, K):
    
    v_0 = eur_option_call_price(r, u, d, s, n, K)   
    
    return v_0


def amer_option_put_price(r, u, d, s, n, K):
    
    def g(s):
        return amer_put(K, s)
    
    v_0 = amer_alg_opt(r, u, d, s, n, g)
    #v_0 = amer_alg_opt_2(r, u, d, s, n, g)
    
    return v_0

И снова, проверка для примера из текстового файла:

In [48]:
res4 = amer_option_put_price(1/4, 2, 1/2, 4, 2, 5)
print(float('{:.7f}'.format(res4)))

1.36


### 7. Примеры

1. Биномиальная модель с 10 периодами. Рассмотрим американский опцион колл со страйком 12. $S_0 = 16$, $u=2$, $d=\dfrac{1}{2}$, $r = \dfrac{1}{4}$.

In [49]:
res5 = amer_option_call_price(1/4, 2, 1/2, 16, 10, 12)
print(float('{:.7f}'.format(res5)))

15.0953001


2. Биномиальная модель с 12 периодами. Рассмотрим европейский опцион пут со страйком 11. $S_0 = 20$, $u=2$, $d=\dfrac{1}{2}$, $r = \dfrac{1}{5}$.

In [50]:
res6 = eur_option_put_price(1/5, 2, 1/2, 20, 12, 11)
print(float('{:.7f}'.format(res6)))

0.4466967


3. Биномиальная модель с 17 периодами. Рассмотрим европейский опцион пут и американский опцион пут со страйком 12. $S_0 = 20$, $u=2$, 

$d=\dfrac{1}{2}$, $r = \dfrac{1}{4}$.

In [165]:
res7 = eur_option_put_price(1/4, 2, 1/2, 20, 17, 12)
print("Европейский опцион")
print(float('{:.7f}'.format(res7)))

res8 = amer_option_put_price(1/4, 2, 1/2, 20, 17, 12)
print("Американский опцион")
print(float('{:.7f}'.format(res8)))

Европейский опцион
0.0834787
Американский опцион
1.7463465


Как мы видим, у европейского опциона пут цена значительно ниже.

4. Биномиальная модель с 17 периодами. Рассмотрим европейский опцион колл и американский опцион колл со страйком 12. $S_0 = 20$, $u=2$, 

$d=\dfrac{1}{2}$, $r = \dfrac{1}{4}$.

In [54]:
res9 = eur_option_call_price(1/4, 2, 1/2, 20, 17, 12)
print("Европейский опцион")
print(float('{:.7f}'.format(res9)))

res10 = amer_option_call_price(1/4, 2, 1/2, 20, 17, 12)
print("Американский опцион")
print(float('{:.7f}'.format(res10)))

def amer_call_12(s):
    return (s - 12)

res11 = amer_alg(1/4, 2, 1/2, 20, 17, amer_call_12)
print("Американский опцион напрямую через американский алгоритм")
print(float('{:.7f}'.format(res10)))

Европейский опцион
19.8132628
Американский опцион
19.8132628
Американский опцион напрямую через американский алгоритм
19.8132628


Как видим, результаты одинаковые.